#### Generating csv with selected users for a given collection 

In [161]:
#############
# Constants (From /Volumes/Untitled/seungwoong.ha/collmind/proj_utils_gpu.py")#
#############
from datetime import datetime

DATA_DIR = "/data/comments/valentin"

COLLECTION_NAMES = ["Atlantic", "Breitbart", "Gatewaypundit", "Motherjones", "Thehill"]

CORRECTED_DICT = {
    "Gatewaypundit": "0",
    "Breitbart": "1",
    "Thehill": "2",
    "Atlantic": "3",
    "Motherjones": "4",
}

CORRECTED_DICT_REVERSE = {
    "0": "Gatewaypundit",
    "1": "Breitbart",
    "2": "Thehill",
    "3": "Atlantic",
    "4": "Motherjones"
}

DATE_RANGES_OLD = {
    "Atlantic": (datetime(2012, 6, 1), datetime(2018, 5, 1)),
    "Breitbart": (datetime(2012, 6, 1), datetime(2021, 6, 1)),
    "Gatewaypundit": (datetime(2015, 1, 1), datetime(2021, 6, 1)),
    "Motherjones": (datetime(2012, 6, 1), datetime(2019, 9, 1)),
    "Thehill": (datetime(2012, 6, 1), datetime(2021, 6, 1))
    
}

DATE_RANGES = {
    "Atlantic": (datetime(2012, 6, 1), datetime(2018, 5, 1)),
    "Breitbart": (datetime(2012, 6, 1), datetime(2023, 4, 1)),
    "Gatewaypundit": (datetime(2015, 1, 1), datetime(2023, 4, 1)),
    "Motherjones": (datetime(2012, 6, 1), datetime(2019, 9, 1)),
    "Thehill": (datetime(2012, 6, 1), datetime(2022, 3, 1))
}

MODEL_NAMES = {
    "Atlantic": 'atlantic_new_s4_r19_h200_u80_t10',
    "Breitbart": 'breitbart_new_s3_r19_h225_u20_t10',
    "Gatewaypundit": 'gatewaypundit_new_s4_r19_h400_u30_t10',
    "Motherjones": 'motherjones_new_s5_r19_h425_u90_t10',
    "Thehill": 'thehill_new_s2_r19_h300_u80_t10',
    "global": 'global_new_s1_r19_h325_u20_t10',
    "title": 'title_new_s2_r19_h35_u50_t10'
}

NUM_TOPICS = {
    "Atlantic": 219,
    "Breitbart": 287,
    "Gatewaypundit": 251,
    "Motherjones": 120,
    "Thehill": 257
}  # excluding -1

In [129]:
import pyarrow.parquet as pq

# Read the schema of the Parquet file
parquet_file = pq.ParquetFile("/Volumes/T7 Shield/user_trajectory_df.parquet")

# Print the schema to inspect the columns and their data types
print(parquet_file.schema)


required group field_id=-1 schema {
  optional binary field_id=-1 user_id (String);
  optional group field_id=-1 comment_id (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 item (String);
    }
  }
  optional group field_id=-1 article_id (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 item (String);
    }
  }
  optional group field_id=-1 collection_id (List) {
    repeated group field_id=-1 list {
      optional int64 field_id=-1 item;
    }
  }
  optional group field_id=-1 createdAt (List) {
    repeated group field_id=-1 list {
      optional int64 field_id=-1 item (Timestamp(isAdjustedToUTC=false, timeUnit=microseconds, is_from_converted_type=false, force_set_converted_type=false));
    }
  }
  optional int64 field_id=-1 num_comments;
  optional int64 field_id=-1 age;
}



In [130]:
import pyarrow.parquet as pq
import pandas as pd

parquet_file = pq.ParquetFile("/Volumes/T7 Shield/user_trajectory_df.parquet")

# Define a reasonable batch size based on your memory constraints
batch_size = 1000
all_dfs = []

for batch in parquet_file.iter_batches(batch_size=batch_size):
    df_batch = batch.to_pandas()
    # Either process each batch individually or collect them
    all_dfs.append(df_batch)

In [131]:
# Optionally combine all batches if you need the complete dataset
full_df = pd.concat(all_dfs, ignore_index=True)
full_df

,user_id,comment_id,article_id,collection_id,createdAt,num_comments,age
0,98984140,"[1296188855, 1538670905, 1538673711, 153867633...","[2479818491, 2919574806, 2919574806, 291957480...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2014-03-22T08:48:35.000000, 2014-08-12T08:07:...",95544,1775
1,119681868,"[1766699258, 1767767286, 1768599205, 176870299...","[3377750257, 3381059643, 3382088630, 338352429...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2015-01-01T00:03:48.000000, 2015-01-01T22:50:...",706,2026
2,80705336,"[1766708109, 1766781995, 1767283930, 176779769...","[3378186944, 3378186944, 3379967272, 337818694...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2015-01-01T00:14:33.000000, 2015-01-01T01:49:...",653,1225
3,46987845,"[848139831, 1318705113, 1353706075, 1399594770...","[1178114936, 2586046961, 2636003224, 270476032...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, ...","[2013-03-31T23:44:39.000000, 2014-04-04T17:14:...",29862,3703
4,137569935,[1766728393],[3377148162],[0],[2015-01-01T00:33:40.000000],1,0
...,...,...,...,...,...,...,...
1992537,386700367,[5945168033],[9303859903],[4],[2022-08-12T06:15:40.000000],1,0
1992538,380909193,[5710779204],[8989006764],[4],[2022-01-27T21:08:27.000000],1,0
1992539,345270447,[5716815397],[8998684079],[4],[2022-02-01T01:18:27.000000],1,0
1992540,381459559,[5730145969],[9013121563],[4],[2022-02-11T13:52:03.000000],1,0


In [159]:
full_df.head(50)

,user_id,comment_id,article_id,collection_id,createdAt,num_comments,age
0,98984140,"[1296188855, 1538670905, 1538673711, 153867633...","[2479818491, 2919574806, 2919574806, 291957480...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2014-03-22T08:48:35.000000, 2014-08-12T08:07:...",95544,1775
1,119681868,"[1766699258, 1767767286, 1768599205, 176870299...","[3377750257, 3381059643, 3382088630, 338352429...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2015-01-01T00:03:48.000000, 2015-01-01T22:50:...",706,2026
2,80705336,"[1766708109, 1766781995, 1767283930, 176779769...","[3378186944, 3378186944, 3379967272, 337818694...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2015-01-01T00:14:33.000000, 2015-01-01T01:49:...",653,1225
3,46987845,"[848139831, 1318705113, 1353706075, 1399594770...","[1178114936, 2586046961, 2636003224, 270476032...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, ...","[2013-03-31T23:44:39.000000, 2014-04-04T17:14:...",29862,3703
4,137569935,[1766728393],[3377148162],[0],[2015-01-01T00:33:40.000000],1,0
5,51937073,"[737260753, 737267800, 737341656, 737342873, 7...","[975751523, 975751523, 975926975, 975926975, 9...","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, ...","[2012-12-15T12:39:44.000000, 2012-12-15T12:48:...",1178,2527
6,32309650,"[1766737650, 2020997030, 2639437267, 279899726...","[3375712419, 3757195202, 4766529149, 500626223...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2015-01-01T00:45:42.000000, 2015-05-12T19:56:...",55,1036
7,121973074,"[1766747894, 1766748778, 1766751269, 176675183...","[3378186944, 3378186944, 3378148626, 337814862...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2015-01-01T00:58:39.000000, 2015-01-01T00:59:...",19,19
8,24825032,"[847134205, 923779608, 923783951, 923805953, 1...","[1173483724, 1369799811, 1369799811, 136979981...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2013-03-30T12:47:51.000000, 2013-06-09T01:01:...",460,3188
9,124660842,"[1646578525, 1705689015, 1706046654, 171030271...","[3138190384, 3247104432, 3247414802, 325862141...","[2, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2014-10-21T16:20:43.000000, 2014-11-21T17:38:...",907,298


In [162]:
# keep only rows that have 3 inside the collection_id column 
collection_name = "Motherjones"
collection_id = int(CORRECTED_DICT[collection_name])
filtered_df = full_df[full_df['collection_id'].apply(lambda x: collection_id in x)]
filtered_df

,user_id,comment_id,article_id,collection_id,createdAt,num_comments,age
3,46987845,"[848139831, 1318705113, 1353706075, 1399594770...","[1178114936, 2586046961, 2636003224, 270476032...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, ...","[2013-03-31T23:44:39.000000, 2014-04-04T17:14:...",29862,3703
5,51937073,"[737260753, 737267800, 737341656, 737342873, 7...","[975751523, 975751523, 975926975, 975926975, 9...","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, ...","[2012-12-15T12:39:44.000000, 2012-12-15T12:48:...",1178,2527
14,40010924,"[841798765, 853566091, 857017277, 859429610, 8...","[1161875980, 1187178720, 1196697494, 120120687...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, ...","[2013-03-25T16:31:39.000000, 2013-04-05T22:56:...",11952,2824
16,36316488,"[871669888, 871673182, 871675634, 871675877, 8...","[1225391695, 1225391695, 1225391695, 122539169...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2013-04-22T01:02:38.000000, 2013-04-22T01:07:...",21758,3672
19,54635532,"[328700828, 522246083, 522247847, 666948162, 8...","[422761156, 679975393, 679975393, 864333042, 1...","[4, 2, 2, 2, 3, 1, 3, 3, 3, 1, 3, 1, 2, 1, 2, ...","[2011-10-07T06:52:53.000000, 2012-05-08T01:43:...",99,4211
...,...,...,...,...,...,...,...
1992537,386700367,[5945168033],[9303859903],[4],[2022-08-12T06:15:40.000000],1,0
1992538,380909193,[5710779204],[8989006764],[4],[2022-01-27T21:08:27.000000],1,0
1992539,345270447,[5716815397],[8998684079],[4],[2022-02-01T01:18:27.000000],1,0
1992540,381459559,[5730145969],[9013121563],[4],[2022-02-11T13:52:03.000000],1,0


In [137]:
# Function to check if all lists in the specified columns are of the same length
def check_list_lengths(row):
    lengths = [len(row['collection_id']), len(row['comment_id']), len(row['article_id']), len(row['createdAt'])]
    return all(length == lengths[0] for length in lengths)

# fucntion to the row by the collection_id
def filter_by_collection_id(row, collection_id):

    collection_name = COLLECTION_DICT_REVERSE[str(collection_id)]
    # Get the mask of indices where the collection_id is equal to the specified value
    mask = [index for index, val in enumerate(row['collection_id']) if val == collection_id]
    
    # Filter each list in the row based on the mask
    row['collection_id'] = [row['collection_id'][i] for i in mask]
    row['comment_id'] = [row['comment_id'][i] for i in mask]
    row['article_id'] = [row['article_id'][i] for i in mask]
    row['createdAt'] = [row['createdAt'][i] for i in mask]

    # re-compute the num_comments (length of the list)
    row[f'num_comments_{collection_name}'] = len(row['comment_id'])

    # Re-compute the age in days
    created_at_series = pd.Series(row['createdAt'])
    max_date = created_at_series.max()
    min_date = created_at_series.min()
    print(f"max_date: {max_date} | min_date: {min_date}")
    age_timedelta =  max_date - min_date
    print(f"age_timedelta: {age_timedelta}")
    row[f'age_{collection_name}'] = age_timedelta.days  # keep only the days

    return row

In [138]:
sub_sample = filtered_df.head(10)
sub_sample.apply(filter_by_collection_id, axis=1, collection_id=collection_id) 

max_date: 2013-09-08 16:00:45 | min_date: 2013-05-17 13:39:48
age_timedelta: 114 days 02:20:57
max_date: 2015-01-14 21:27:58 | min_date: 2014-01-21 04:37:29
age_timedelta: 358 days 16:50:29
max_date: 2018-01-15 23:09:41 | min_date: 2013-05-08 22:53:11
age_timedelta: 1713 days 00:16:30
max_date: 2015-05-16 05:03:05 | min_date: 2014-07-28 16:46:02
age_timedelta: 291 days 12:17:03
max_date: 2017-09-05 17:26:12 | min_date: 2014-12-30 05:01:25
age_timedelta: 980 days 12:24:47
max_date: 2017-10-16 00:13:38 | min_date: 2013-04-02 07:20:31
age_timedelta: 1657 days 16:53:07
max_date: 2015-01-17 17:38:25 | min_date: 2015-01-17 17:38:25
age_timedelta: 0 days 00:00:00
max_date: 2017-04-20 17:36:06 | min_date: 2013-12-17 14:24:26
age_timedelta: 1220 days 03:11:40
max_date: 2015-04-28 20:22:27 | min_date: 2011-07-17 16:42:58
age_timedelta: 1381 days 03:39:29
max_date: 2016-08-09 11:26:03 | min_date: 2012-03-13 01:02:04
age_timedelta: 1610 days 10:23:59


,user_id,comment_id,article_id,collection_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones
5,51937073,"[900304527, 1034338827]","[1294139038, 1722649092]","[3, 3]","[2013-05-17T13:39:48.000000, 2013-09-08T16:00:...",1178,2527,2,114
13,17337383,"[1210056354, 1792883300]","[2148686575, 3419704334]","[3, 3]","[2014-01-21T04:37:29.000000, 2015-01-14T21:27:...",229,1712,2,358
16,36316488,"[890113275, 890116470, 890117408, 892685212, 8...","[1270210643, 1270210643, 1270210643, 127021064...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2013-05-08T22:53:11.000000, 2013-05-08T22:58:...",21758,3672,148,1713
17,8943013,"[1508393989, 2028248432]","[2870238432, 3765980706]","[3, 3]","[2014-07-28T16:46:02.000000, 2015-05-16T05:03:...",303,3461,2,291
18,119366679,"[1764128603, 1764144474, 1765029723, 176650388...","[3368547807, 3368547807, 3368547807, 337706455...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2014-12-30T05:01:25.000000, 2014-12-30T05:24:...",5347,3096,104,980
19,54635532,"[849229436, 1105807941, 1119751730, 1136654918...","[1170747567, 1926642194, 1956744486, 198875945...","[3, 3, 3, 3, 3, 3, 3, 3]","[2013-04-02T07:20:31.000000, 2013-11-02T01:16:...",99,4211,8,1657
20,26906017,[1798520331],[3423145942],[3],[2015-01-17T17:38:25.000000],226,1847,1,0
26,27844421,"[1167432204, 2287916431, 3265207729]","[1839881688, 4187245383, 5736691121]","[3, 3, 3]","[2013-12-17T14:24:26.000000, 2015-10-03T15:13:...",1850,2889,3,1220
31,45967332,"[255003571, 802094229, 1180337165, 1180349056,...","[359125574, 1085644206, 2071098652, 2071098652...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[2011-07-17T16:42:58.000000, 2013-02-17T05:16:...",61,2370,13,1381
33,22885995,"[463862502, 526441588, 1379061397, 1379061863,...","[608200902, 682298191, 1147501231, 1147501231,...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[2012-03-13T01:02:04.000000, 2012-05-11T16:14:...",21071,2495,11,1610


In [70]:
# it took 3 minutes! 
collection_name = "Motherjones"
collection_id = int(COLLECTION_DICT[collection_name])
motherjones_df = filtered_df.apply(filter_by_collection_id, axis=1, collection_id=collection_id) 
motherjones_df

,user_id,comment_id,article_id,collection_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones
5,51937073,"[900304527, 1034338827]","[1294139038, 1722649092]","[3, 3]","[2013-05-17T13:39:48.000000, 2013-09-08T16:00:...",1178,2527,2,114
13,17337383,"[1210056354, 1792883300]","[2148686575, 3419704334]","[3, 3]","[2014-01-21T04:37:29.000000, 2015-01-14T21:27:...",229,1712,2,358
16,36316488,"[890113275, 890116470, 890117408, 892685212, 8...","[1270210643, 1270210643, 1270210643, 127021064...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2013-05-08T22:53:11.000000, 2013-05-08T22:58:...",21758,3672,148,1713
17,8943013,"[1508393989, 2028248432]","[2870238432, 3765980706]","[3, 3]","[2014-07-28T16:46:02.000000, 2015-05-16T05:03:...",303,3461,2,291
18,119366679,"[1764128603, 1764144474, 1765029723, 176650388...","[3368547807, 3368547807, 3368547807, 337706455...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2014-12-30T05:01:25.000000, 2014-12-30T05:24:...",5347,3096,104,980
...,...,...,...,...,...,...,...,...,...
1867515,345229335,[5681874501],[8962168999],[3],[2022-01-08T19:39:45.000000],1,0,1,0
1867516,396832875,[6249694848],[4412245423],[3],[2023-08-07T21:27:39.000000],1,0,1,0
1867517,391756133,[6271694238],[231276105],[3],[2023-09-05T23:19:13.000000],1,0,1,0
1867518,381429732,[6123593697],[208754508],[3],[2023-02-25T01:13:52.000000],1,0,1,0


In [142]:
df = motherjones_df.copy()
df.drop(columns='collection_id', inplace=True)
df

,user_id,comment_id,article_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones
5,51937073,"[900304527, 1034338827]","[1294139038, 1722649092]","[2013-05-17T13:39:48.000000, 2013-09-08T16:00:...",1178,2527,2,114
13,17337383,"[1210056354, 1792883300]","[2148686575, 3419704334]","[2014-01-21T04:37:29.000000, 2015-01-14T21:27:...",229,1712,2,358
16,36316488,"[890113275, 890116470, 890117408, 892685212, 8...","[1270210643, 1270210643, 1270210643, 127021064...","[2013-05-08T22:53:11.000000, 2013-05-08T22:58:...",21758,3672,148,1713
17,8943013,"[1508393989, 2028248432]","[2870238432, 3765980706]","[2014-07-28T16:46:02.000000, 2015-05-16T05:03:...",303,3461,2,291
18,119366679,"[1764128603, 1764144474, 1765029723, 176650388...","[3368547807, 3368547807, 3368547807, 337706455...","[2014-12-30T05:01:25.000000, 2014-12-30T05:24:...",5347,3096,104,980
...,...,...,...,...,...,...,...,...
1867515,345229335,[5681874501],[8962168999],[2022-01-08T19:39:45.000000],1,0,1,0
1867516,396832875,[6249694848],[4412245423],[2023-08-07T21:27:39.000000],1,0,1,0
1867517,391756133,[6271694238],[231276105],[2023-09-05T23:19:13.000000],1,0,1,0
1867518,381429732,[6123593697],[208754508],[2023-02-25T01:13:52.000000],1,0,1,0


In [146]:
def check_and_clean_lengths(df: pd.DataFrame) -> pd.DataFrame:
    """
    Checks if each row has consistent lengths across specified fields and drops length columns.
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        
    Returns:
        pd.DataFrame: Cleaned DataFrame with length columns removed
    """
    # Fields to check
    fields = ['comment_id', 'createdAt', 'article_id']
    
    # Create length columns efficiently using vectorized operations
    for field in fields:
        df[f'{field}_length'] = df[field].apply(len)
    
    # Check if each row has consistent lengths across fields
    length_columns = [f'{field}_length' for field in fields]
    inconsistent_rows = df[length_columns].nunique(axis=1) > 1
    
    if not inconsistent_rows.any():
        print("All rows have consistent lengths across fields")
    else:
        print(f"Warning: {inconsistent_rows.sum()} rows have inconsistent lengths across fields")
        # Print the first few rows with inconsistent lengths
        inconsistent_data = df[inconsistent_rows].head()
        for idx, row in inconsistent_data.iterrows():
            print(f"\nRow {idx} has different lengths:")
            for field in fields:
                print(f"{field}: {row[f'{field}_length']}")
    
    # Drop length columns
    df = df.drop(columns=length_columns)
    
    return df

check_and_clean_lengths(df)

All rows have consistent lengths across fields


,user_id,comment_id,article_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones
5,51937073,"[900304527, 1034338827]","[1294139038, 1722649092]","[2013-05-17T13:39:48.000000, 2013-09-08T16:00:...",1178,2527,2,114
13,17337383,"[1210056354, 1792883300]","[2148686575, 3419704334]","[2014-01-21T04:37:29.000000, 2015-01-14T21:27:...",229,1712,2,358
16,36316488,"[890113275, 890116470, 890117408, 892685212, 8...","[1270210643, 1270210643, 1270210643, 127021064...","[2013-05-08T22:53:11.000000, 2013-05-08T22:58:...",21758,3672,148,1713
17,8943013,"[1508393989, 2028248432]","[2870238432, 3765980706]","[2014-07-28T16:46:02.000000, 2015-05-16T05:03:...",303,3461,2,291
18,119366679,"[1764128603, 1764144474, 1765029723, 176650388...","[3368547807, 3368547807, 3368547807, 337706455...","[2014-12-30T05:01:25.000000, 2014-12-30T05:24:...",5347,3096,104,980
...,...,...,...,...,...,...,...,...
1867515,345229335,[5681874501],[8962168999],[2022-01-08T19:39:45.000000],1,0,1,0
1867516,396832875,[6249694848],[4412245423],[2023-08-07T21:27:39.000000],1,0,1,0
1867517,391756133,[6271694238],[231276105],[2023-09-05T23:19:13.000000],1,0,1,0
1867518,381429732,[6123593697],[208754508],[2023-02-25T01:13:52.000000],1,0,1,0


In [ ]:
sub_sample = df[(df['num_comments_Motherjones'] > 1000) & (df['age_Motherjones'] >= 365)].copy()

In [81]:
# 1.2 Selct longest active users and with at least 100 comments 
percentile_95_time = df["age_Motherjones"].quantile(0.95)
df_top5_time = df[df["age_Motherjones"] >= percentile_95_time]
sub_sample_df = df_top5_time[df_top5_time['num_comments_Motherjones'] >= 100]

sub_sample_ids = sub_sample_df['user_id'].tolist()
print(f"Computed sub sample ids. Users are {len(sub_sample_ids)}")


Computed sub sample ids. Users are 3815


In [82]:
top_old_users = sub_sample_df.copy()

In [89]:
top_old_users.head(1)

,user_id,comment_id,article_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones
16,36316488,"[890113275, 890116470, 890117408, 892685212, 8...","[1270210643, 1270210643, 1270210643, 127021064...","[2013-05-08T22:53:11.000000, 2013-05-08T22:58:...",21758,3672,148,1713


In [113]:

# Function to calculate monthly comment statistics
def calculate_monthly_stats(row):
    # Sort the 'createdAt' dates
    sorted_dates = sorted(row['createdAt'])
    
    # Convert to year-month format
    year_month = pd.to_datetime(sorted_dates).to_period('M')
    
    # Count comments per month
    monthly_counts = year_month.value_counts().sort_index()
    
    # Create a complete range of months from the first to the last comment
    all_months = pd.period_range(start=year_month.min(), end=year_month.max(), freq='M')
    
    # Reindex the monthly counts to include all months, filling missing months with 0
    monthly_counts = monthly_counts.reindex(all_months, fill_value=0)
    
    # Calculate mean and std of monthly comments
    mean_comments = monthly_counts.mean()
    std_comments = monthly_counts.std()
    
    # Count the number of active months
    active_months = len(monthly_counts)
    
    row['mean_comments_per_month'] = mean_comments
    row['std_comments_per_month'] = std_comments
    row['months_counted'] = active_months
    
    return row

In [155]:
from tqdm import tqdm
tqdm.pandas(desc="Processing Users")
sub_sample = sub_sample.progress_apply(calculate_monthly_stats, axis=1)


Processing Users: 100%|██████████| 1117/1117 [00:03<00:00, 280.04it/s]


In [156]:
sub_sample

,user_id,comment_id,article_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones,comment_id_length,createdAt_length,article_id_length,mean_comments_per_month,std_comments_per_month,months_counted
302,3927078,"[1354032435, 1831544320, 1832387892, 185166925...","[2616613440, 3473611290, 3479283861, 351229219...","[2014-04-24T19:47:46.000000, 2015-02-03T00:23:...",2518,3803,2228,951,2228,2228,2228,67.515152,113.014690,33
569,160339,"[500492780, 500494063, 729162994, 729164657, 7...","[208502675, 208502675, 959429052, 959429052, 9...","[2012-04-18T04:49:59.000000, 2012-04-18T04:52:...",35189,4051,3604,2120,3604,3604,3604,50.760563,75.049786,71
622,116787800,"[1680815728, 1732764427, 1737665478, 173766649...","[3197877410, 3302969459, 3307521133, 330752113...","[2014-11-07T04:20:31.000000, 2014-12-09T05:17:...",2022,1516,1447,1016,1447,1447,1447,42.558824,44.066198,34
735,986307,"[1997440341, 2421347877, 2421349477, 242135152...","[3722842075, 4423853496, 4423853496, 442385349...","[2015-05-01T10:11:38.000000, 2015-12-22T12:57:...",8001,2989,2043,1012,2043,2043,2043,60.088235,68.566874,34
751,75217241,"[1062839134, 1062844105, 1062857803, 106285953...","[1801861366, 1801861366, 1801861366, 180186136...","[2013-09-28T20:06:24.000000, 2013-09-28T20:12:...",3672,2945,1043,1558,1043,1043,1043,19.679245,44.213109,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716023,7771630,"[3076407740, 3076411082, 3076417749, 307659392...","[5423225297, 4929964184, 4777873281, 542322529...","[2016-12-31T03:43:19.000000, 2016-12-31T03:47:...",1851,399,1851,399,1851,1851,1851,123.400000,78.553712,15
1718117,36737813,"[3123750865, 3123753296, 3123770886, 312378033...","[5498996535, 5498996535, 5498996535, 549899653...","[2017-01-27T20:08:51.000000, 2017-01-27T20:10:...",9439,376,9439,376,9439,9439,9439,674.214286,320.606439,14
1775863,23775273,"[469634235, 469645844, 469647236, 469658092, 4...","[616950603, 613761190, 613761190, 613761190, 6...","[2012-03-20T02:28:22.000000, 2012-03-20T02:42:...",3117,379,3117,379,3117,3117,3117,222.642857,135.573771,14
1777220,2222178,"[47488209, 67975098, 68004597, 68016063, 68016...","[90632809, 128531418, 128531418, 128531418, 12...","[2010-04-29T19:53:37.000000, 2010-08-11T20:02:...",1075,1132,1075,1132,1075,1075,1075,27.564103,54.700041,39


In [157]:
sub_sample['cv_comments_per_month'] = (
        sub_sample['std_comments_per_month'] / sub_sample['mean_comments_per_month']
    )
sub_sample

,user_id,comment_id,article_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones,comment_id_length,createdAt_length,article_id_length,mean_comments_per_month,std_comments_per_month,months_counted,cv_comments_per_month
302,3927078,"[1354032435, 1831544320, 1832387892, 185166925...","[2616613440, 3473611290, 3479283861, 351229219...","[2014-04-24T19:47:46.000000, 2015-02-03T00:23:...",2518,3803,2228,951,2228,2228,2228,67.515152,113.014690,33,1.673916
569,160339,"[500492780, 500494063, 729162994, 729164657, 7...","[208502675, 208502675, 959429052, 959429052, 9...","[2012-04-18T04:49:59.000000, 2012-04-18T04:52:...",35189,4051,3604,2120,3604,3604,3604,50.760563,75.049786,71,1.478506
622,116787800,"[1680815728, 1732764427, 1737665478, 173766649...","[3197877410, 3302969459, 3307521133, 330752113...","[2014-11-07T04:20:31.000000, 2014-12-09T05:17:...",2022,1516,1447,1016,1447,1447,1447,42.558824,44.066198,34,1.035419
735,986307,"[1997440341, 2421347877, 2421349477, 242135152...","[3722842075, 4423853496, 4423853496, 442385349...","[2015-05-01T10:11:38.000000, 2015-12-22T12:57:...",8001,2989,2043,1012,2043,2043,2043,60.088235,68.566874,34,1.141103
751,75217241,"[1062839134, 1062844105, 1062857803, 106285953...","[1801861366, 1801861366, 1801861366, 180186136...","[2013-09-28T20:06:24.000000, 2013-09-28T20:12:...",3672,2945,1043,1558,1043,1043,1043,19.679245,44.213109,53,2.246687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716023,7771630,"[3076407740, 3076411082, 3076417749, 307659392...","[5423225297, 4929964184, 4777873281, 542322529...","[2016-12-31T03:43:19.000000, 2016-12-31T03:47:...",1851,399,1851,399,1851,1851,1851,123.400000,78.553712,15,0.636578
1718117,36737813,"[3123750865, 3123753296, 3123770886, 312378033...","[5498996535, 5498996535, 5498996535, 549899653...","[2017-01-27T20:08:51.000000, 2017-01-27T20:10:...",9439,376,9439,376,9439,9439,9439,674.214286,320.606439,14,0.475526
1775863,23775273,"[469634235, 469645844, 469647236, 469658092, 4...","[616950603, 613761190, 613761190, 613761190, 6...","[2012-03-20T02:28:22.000000, 2012-03-20T02:42:...",3117,379,3117,379,3117,3117,3117,222.642857,135.573771,14,0.608929
1777220,2222178,"[47488209, 67975098, 68004597, 68016063, 68016...","[90632809, 128531418, 128531418, 128531418, 12...","[2010-04-29T19:53:37.000000, 2010-08-11T20:02:...",1075,1132,1075,1132,1075,1075,1075,27.564103,54.700041,39,1.984467


In [158]:
sub_sample.sort_values(by='cv_comments_per_month', ascending=True).head(10)

,user_id,comment_id,article_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones,comment_id_length,createdAt_length,article_id_length,mean_comments_per_month,std_comments_per_month,months_counted,cv_comments_per_month
420807,37386588,"[719787272, 719795168, 719796614, 720639694, 7...","[945273586, 944878360, 944878360, 946849456, 9...","[2012-11-27T00:26:18.000000, 2012-11-27T00:41:...",1847,883,1844,883,1844,1844,1844,61.466667,20.143850,30,0.327720
754656,239181551,"[3105839671, 3107719066, 3107730469, 310773823...","[5471439454, 5475434979, 5474248859, 547543497...","[2017-01-18T14:04:22.000000, 2017-01-19T12:43:...",13914,1867,6189,389,6189,6189,6189,442.071429,149.375348,14,0.337899
1686872,199536272,"[2556258911, 2556296888, 2556309207, 255632655...","[4641479276, 4641479276, 4641479276, 464147927...","[2016-03-07T17:02:34.000000, 2016-03-07T17:24:...",9835,701,9835,701,9835,9835,9835,409.791667,149.506137,24,0.364834
1553366,3318757,"[56335467, 56337116, 56578598, 56685614, 57906...","[106641791, 106641791, 106641791, 107652224, 1...","[2010-06-12T03:32:56.000000, 2010-06-12T03:55:...",1114,838,1114,838,1114,1114,1114,39.785714,15.149519,28,0.380778
1667279,40009685,"[2244574111, 2248202405, 2248214841, 224904309...","[4112832676, 4119508648, 4119508648, 411950864...","[2015-09-09T16:25:05.000000, 2015-09-11T15:22:...",5594,881,5594,881,5594,5594,5594,186.466667,75.505530,30,0.404928
90614,234592611,"[3048295423, 3048633270, 3048685478, 304869291...","[5364547079, 5374879711, 5374879711, 537487971...","[2016-12-12T14:35:42.000000, 2016-12-12T18:07:...",19136,2008,8327,425,8327,8327,8327,555.133333,225.691910,15,0.406554
52295,147233829,"[1896619363, 1896625398, 1896626721, 189663225...","[3579896704, 3579896704, 3579896704, 357989670...","[2015-03-09T13:58:37.000000, 2015-03-09T14:02:...",61263,1712,26133,690,26133,26133,26133,1136.217391,463.505120,23,0.407937
1553452,21166113,"[410287508, 410290372, 410292066, 410301992, 4...","[537870817, 537870817, 537870817, 537535121, 5...","[2012-01-14T02:00:47.000000, 2012-01-14T02:06:...",12048,2215,12047,2215,12047,12047,12047,162.797297,68.243516,74,0.419193
427713,123045080,"[1623513703, 1623616771, 1623642712, 162371896...","[3091977225, 3091977225, 3091977225, 309197722...","[2014-10-07T14:26:01.000000, 2014-10-07T15:33:...",13987,1396,13545,1219,13545,13545,13545,330.365854,144.410311,41,0.437122
1298563,173145548,"[2332019832, 2332027939, 2332032026, 234874535...","[4267825990, 4267825990, 4267825990, 430032812...","[2015-10-28T22:18:16.000000, 2015-10-28T22:23:...",4834,641,4734,410,4734,4734,4734,315.600000,139.531768,15,0.442116


In [125]:
top_old_users.sort_values(by='cv_comments_per_month', ascending=True).head(1000)["user_id"]

1553452     21166113
427713     123045080
1633761    134099972
405252      38075423
77635       18125924
             ...    
85453       74034112
42411       64966666
5473         2830525
1553903     46463844
1618420    115676977
Name: user_id, Length: 1000, dtype: object

In [127]:
top_old_users[top_old_users['user_id'] == "80135701"]

,user_id,comment_id,article_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones,mean_comments_per_month,std_comments_per_month,months_counted,cv_comments_per_month
666390,80135701,"[1106572745, 1127624244, 1202099421, 141822404...","[1926909973, 1968819126, 2125829715, 273338403...","[2013-11-02T20:46:07.000000, 2013-11-18T03:49:...",413,1551,409,1551,7.865385,9.46413,52,1.203263


In [123]:

# save top 1k users 

output_path = "/Users/barolo/Desktop/PhD/Code/Comments_Beliefs/Batch_calling/data/selected_users/motherjones.csv"


top_old_users.sort_values(by='cv_comments_per_month', ascending=True).head(1000)["user_id"].to_csv(output_path, index=False)



In [135]:
longest_active_users = pd.read_csv("/Users/barolo/Desktop/PhD/Code/Comments_Beliefs/selected_users/motherjones_longest_active_users.csv")
longest_active_users

,user_id,comment_id,article_id,collection_id,createdAt,num_comments,age,num_comments_Motherjones,age_Motherjones
0,36316488,"['890113275', '890116470', '890117408', '89268...","['1270210643', '1270210643', '1270210643', '12...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2013-05-08T22:53:11.000000'), ...",21758,3672,148,1713
1,30204007,"['643680515', '643681055', '643681863', '93492...","['821899250', '821899250', '821899250', '14109...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2012-09-07T22:06:02.000000'), ...",1991,3923,312,1959
2,24419220,"['545971795', '569053573', '614453908', '65983...","['709803868', '741517009', '789930633', '85248...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2012-06-02T22:32:24.000000'), ...",11158,4013,175,1999
3,17601626,"['360182846', '360864575', '387765463', '38932...","['466758712', '467547282', '505526945', '50806...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2011-11-10T17:48:01.000000'), ...",7928,3169,224,1260
4,6172172,"['137231851', '140613133', '282198196', '28220...","['215520142', '221725692', '381172630', '38149...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2011-01-30T15:48:19.000000'), ...",827,4026,108,2563
...,...,...,...,...,...,...,...,...,...
3810,39851504,"['750480023', '3157258373', '3159796814', '315...","['995666061', '5553846599', '5557463979', '555...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2012-12-29T11:45:29.000000'), ...",395,1865,395,1865
3811,3745567,"['344040067', '644573640', '714463185', '71447...","['452427397', '833582334', '935266256', '93526...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2011-10-25T19:38:42.000000'), ...",180,2289,157,2289
3812,20296533,"['389185537', '415587195', '415594949', '41667...","['506733396', '545447168', '545447168', '54544...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2011-12-18T13:20:58.000000'), ...",558,2242,558,2242
3813,33178489,"['36816176', '36817668', '36821613', '66517140...","['70259489', '70259603', '70259930', '86198301...","[np.int64(3), np.int64(3), np.int64(3), np.int...","[np.datetime64('2010-02-08T17:36:38.000000'), ...",112,1249,112,1249
